In [ ]:
!sudo pip install catboost

# Обработка лидарных данных
## Сегментация

### Про лидар

![](https://realnoevremya.com/uploads/gallery/b9/7e/35d10da899958110.jpg)

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('Pa-q5elS_nE')

## А что за данные на самом деле

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import tqdm
py.init_notebook_mode(connected=True)

EQUAL_ASPECT_RATIO_LAYOUT = dict(
    margin={
        'l': 0,
        'r': 0,
        'b': 0,
        't': 0
    }, scene=dict(
    aspectmode='data'
))


def color(x, cmap='Reds'):
    cmap = plt.get_cmap(cmap)
    x = (x - np.min(x)) / np.max(x)
    
    return cmap(x)

%matplotlib inline

In [ ]:
ds = pd.read_csv('./snow.csv')
ds = ds.set_index(['scene_id'])
ds.head()

* intensity - ???
* ring - ???

### Кольцо

![](https://eckop.com/wp-content/uploads/2018/07/LIDAR_02-1024x366.png)

In [ ]:
scene = ds.loc[0]

fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene.ring, 'tab20'),
    },
    'text': scene.ring
})

py.iplot(fig)


### Интенсивность

![](https://res.mdpi.com/sensors/sensors-15-28099/article_deploy/html/images/sensors-15-28099-g001-1024.png)

In [ ]:
fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene.intensity, 'seismic'),
    },
    'text': scene.intensity
})

py.iplot(fig)

In [ ]:
scene = ds.loc[1]

fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene.intensity, 'seismic'),
    },
    'text': scene.ring
})

py.iplot(fig)


# Отфильтруем снег

## Эвристикой

In [ ]:
def filter_by_intensity(intensity, limit=3):
    return intensity > limit

filtered_scene = scene[filter_by_intensity(scene.intensity)]


fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': filtered_scene.x,
    'y': filtered_scene.y,
    'z': filtered_scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(filtered_scene.intensity, 'seismic'),
    },
    'text': scene.ring
})

py.iplot(fig)

Плохо и непонятно, будем учить

## Облачные вычисления

In [ ]:
from sklearn.neighbors import KDTree

class ComputeFeatures(object):
    def __init__(self, r=1.0):
        self.xyz = None
        self.intensity = None
        self.ring = None
        self.index = None
        self.r = r
        
    def compute_point_features(self, point_id, neighbours):
        
        if len(neighbours) > 0:
            xyz_n, intensity_n, ring_n = self.xyz[neighbours], self.intensity[neighbours], self.ring[neighbours]

            rel_xyz = xyz_n - self.xyz[point_id][np.newaxis, :]
            rel_r = np.linalg.norm(rel_xyz, axis=1) / np.linalg.norm(self.xyz[point_id])
        
            return [
                np.min(intensity_n), np.max(intensity_n), np.median(intensity_n), np.std(intensity_n), 
                np.min(ring_n), np.max(ring_n), np.median(ring_n), np.std(ring_n),
                np.std(rel_r),
                len(neighbours),
                self.intensity[point_id], self.ring[point_id]
            ]
        else:
            return [
                0, 0, 0, 0,
                0, 0, 0, 0,
                0,
                len(neighbours),
                self.intensity[point_id], self.ring[point_id]
            ]
    
    def get_point_neighbours(self, point_id):
        return self.index.query_radius(self.xyz[point_id][np.newaxis, :], r=self.r)[0]
        
    def __call__(self, xyz, intensity, ring):
        self.xyz = xyz[:]
        self.intensity = intensity[:]
        self.ring = ring[:]
        
        self.index = KDTree(self.xyz)
        
        features = []
        for point_id in range(len(self.xyz)):
            neighbours = self.get_point_neighbours(point_id)
            features.append(self.compute_point_features(point_id, neighbours))
        
        
        return pd.DataFrame(columns=self._feature_names(), data=features)
    
    def _feature_names(self):
        return [x.format(self.r) for x in [
            "min_intensity_{}", "max_intensity_{}", "median_intensity_{}", "std_intensity_{}",
            "min_ring_{}", "max_ring_{}", "median_ring_{}", "std_ring_{}",
            "r_std_{}",
            "n_neighbours_{}"]] + ["intensity", "ring"]


In [ ]:
ds_features = pd.read_csv('./snow_features.csv')
ds_features = ds_features.drop(["Unnamed: 0"], axis=1)
ds_features.shape

In [ ]:
features = ComputeFeatures(r=1.0)

for scene_id in tqdm.tqdm(ds.reset_index().scene_id.unique()):
    scene = ds.loc[scene_id]
    features_df = \
        features(scene[['x', 'y', 'z']].values, scene.intensity.values, scene.ring.values)
    features_df.to_csv('./features/{}.csv'.format(scene_id))

# Посмотрим на разметку

In [ ]:
scene = ds.loc[1]

fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene.label, 'seismic'),
    },
    'text': scene.label
})

py.iplot(fig)

## Поучим что-нибудь

In [ ]:
all_scenes = ds_features.scene_id.unique()

train_scenes = np.random.choice(all_scenes, replace=False, size=int(0.8 * len(all_scenes)))
not_train_scenes = np.array([scene_id for scene_id in all_scenes if scene_id not in train_scenes])

test_scenes = np.random.choice(not_train_scenes, replace=False, size=int(0.1 * len(all_scenes)))
val_scenes = [scene_id for scene_id in not_train_scenes if scene_id not in test_scenes]

assert all([scene not in train_scenes for scene in val_scenes])
assert all([scene not in train_scenes for scene in test_scenes])
assert all([scene not in test_scenes for scene in val_scenes])


In [ ]:
train = ds_features[ds_features.scene_id.isin(train_scenes)]
test = ds_features[ds_features.scene_id.isin(test_scenes)]
val = ds_features[ds_features.scene_id.isin(val_scenes)]

In [ ]:
import catboost

def learn(X_train, X_val, y_train, y_val):
    clf = catboost.CatBoostClassifier(n_estimators=100)
    clf.fit(
        X_train, y_train, early_stopping_rounds=10,
        use_best_model=True, eval_set=(X_val.values, y_val.values), plot=True, verbose=False)
    return clf

X_train = train.drop(["scene_id", "label", "x", "y", "z"], axis=1)
y_train = train.label


X_val = val.drop(["scene_id", "label", "x", "y", "z"], axis=1)
y_val = val.label

In [ ]:
#del ds
cls = learn(X_train, X_val, y_train, y_val)


In [ ]:
X_test = test.drop(['scene_id', 'x', 'y', 'z', 'label'], axis=1)
y_test = test.label

from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from rosbag_utils.cloud.filters import filter_snow_indices_in_array

def test_one(clf, X_test, y_test):
    y_test_hat = clf.predict_proba(X_test)
    pr, rec, thr = precision_recall_curve(y_test, y_test_hat[:, 1])
    ix = np.linspace(1, len(pr)-1, num=2000).astype(int)
    return pr[ix], rec[ix], thr[ix - 1]


def heuristic_filter_scoring():
    pr = []
    rec = []
    filter_range = range(1, 10)
    for i in filter_range:
        y_test_heuristic_hat = np.ones(len(X_test))
        y_test_heuristic_hat[filter_by_intensity(test.intensity, i)] = 0
        pr.append(precision_score(y_test, y_test_heuristic_hat))
        rec.append(recall_score(y_test, y_test_heuristic_hat))
        
    return pr, rec, filter_range

pr_bl, rec_bl, thr_bl = heuristic_filter_scoring()

def plot_pr_rec(*models):
    traces = []
    for model, clf, X_test, y_test in models:
        pr, rec, thr = test_one(clf, X_test, y_test)
        pr_rec = go.Scattergl(x = rec, y = pr, mode='lines', text=thr, name=model)
        traces.append(pr_rec)

    pr_rec_bl = go.Scatter(x = rec_bl, y = pr_bl, mode='lines+markers', text=thr_bl, name='Intensity BL')

    layout = go.Layout(
        title='Precission-recall',
        xaxis=dict(
            title='Recall'
        ),
        yaxis=dict(
            title='Precission'
        ))
    fig = go.Figure(
        data=traces + [pr_rec_bl],
        layout=layout)
    py.iplot(fig)
    
models = [('Catboost classifier', cls, X_test, y_test)]
plot_pr_rec(*models)

# Повизуализируем

In [ ]:
y_test_hat = cls.predict_proba(test.drop(['scene_id', 'x', 'y', 'z', 'label'], axis=1))

In [ ]:
array([  1.,  22.,  24.,  27.,  40.,  47.,  57.,  66.,  67.,  76.,  87.,
        93.,  97., 101., 102., 113., 126., 154., 167., 170., 182., 212.,
       237., 240., 248., 249., 259., 274., 285.])

In [ ]:
itest = test.set_index(['scene_id'])

In [ ]:
scene_id = 212
scene = itest.loc[scene_id]
scene_predictions = y_test_hat[test.scene_id == scene_id][:, 1]



In [ ]:

fig = go.Figure(layout=EQUAL_ASPECT_RATIO_LAYOUT)
fig.add_scatter3d(**{
    'x': scene.x,
    'y': scene.y,
    'z': scene.z,
    'mode': 'markers',
    'marker': {
        'size': 1,
        'color': color(scene_predictions, 'seismic'),
    },
    'text': scene_predictions
})

py.iplot(fig)